<a href="https://colab.research.google.com/github/cri5Castro/ASR/blob/master/Interactions_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Dependencies
%%capture
!pip install git+https://github.com/huggingface/transformers.git

In [2]:
from glob import glob
from tqdm.notebook import tqdm
from google.colab import drive
import os
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import cycle
cycol = cycle('bgrcmk')
drive.mount('/content/drive')
root_path='/content/drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
interactions_calls = {os.path.split(filename)[-1]: pd.read_pickle(filename) for filename in glob(f"{root_path}/*/*/*.pkl")}

In [4]:
interactions_calls

{'cases_transcriptions_VM.pkl':     user_id            rfc  ...  vad_n  prob_speech
 0    273858  MAMC760324DN4  ...     18     1.000000
 1   1036900  CAYA7607122V4  ...      6     1.000000
 2   1036900  CAYA7607122V4  ...     85     0.964706
 3    273858  MAMC760324DN4  ...      1     0.000000
 4   1036900  CAYA7607122V4  ...      1     0.000000
 5    273858  MAMC760324DN4  ...     11     0.909091
 6   1036900  CAYA7607122V4  ...     21     0.714286
 7   1036900  CAYA7607122V4  ...      1     0.000000
 8    273858  MAMC760324DN4  ...      0          NaN
 9    273858  MAMC760324DN4  ...      0          NaN
 10   273858  MAMC760324DN4  ...      0          NaN
 11  1036900  CAYA7607122V4  ...      0          NaN
 
 [12 rows x 51 columns],
 'opportunities_transcriptions_VM.pkl':     user_id            rfc  ...  vad_n  prob_speech
 0    273858  MAMC760324DN4  ...     18     1.000000
 1    273858  MAMC760324DN4  ...     18     1.000000
 2    273858  MAMC760324DN4  ...     39     1.000000
 3

In [5]:
interactions_calls['cases_transcriptions_VM.pkl']

,user_id,rfc,natural_person_id,application_answers_id,loan_id,status,approved_date,case_des_status,case_des_origin,case_des_subject,case_date_createddate,case_date_closeddate,case_des_casenumber,case_des_origin.1,case_des_description,task_date_createddate,task_des_status,task_des_priority,task_des_description,task_des_type,task_des_calltype,task_des_callobject,task_des_tasksubtype,date_execution,num_agent_after_contact_work_duration,date_agent_after_contact_work_end,date_agent_after_contact_work_start,num_agent_interaction_duration,date_connected_to_agent,des_agent_username,num_agent_number_of_holds,des_agent_routing_profile_name,num_agent_connection_attempts,des_channel,id_contact,num_customer_phone_number,num_system_phone_number,date_disconnect,date_initiation,id_initiation_contact,id_next_contact,id_previous_contact,des_initiation_method,des_queue_name,date_created,ctr_medialocation,transcript,path,vad_prob,vad_n,prob_speech
0,273858,MAMC760324DN4,273540,1323378,43742,E,2020-06-25,Closed,Phone,Top Up,2020-07-13 11:41:45,2020-08-04 16:51:57,72050,Phone,NaN,2020-08-03 16:43:46,Completed,Normal,NaN,Call,Outbound,19617d8f-9639-491b-9645-e161401d55f2,Call,20200825.0,2.0,2020-08-03 21:45:35,2020-08-03 21:45:33,109.0,2020-08-03 21:43:44,frida.delvalle,0.0,Customer Happiness,0.0,VOICE,19617d8f-9639-491b-9645-e161401d55f2,5.266565e+11,5.255416e+11,2020-08-03 21:45:33,2020-08-03 21:43:28,NaN,NaN,NaN,OUTBOUND,Customer Success,2020-08-25 16:03:58,"""connect-4ea90fb20d17/connect/connect-konfio/C...","[la gripa, None, None, None, None, Disculpa es...",connect-4ea90fb20d17/connect/connect-konfio/Ca...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",18,1.000000
1,1036900,CAYA7607122V4,1069028,1328804,43839,V,2020-07-06,Closed,Phone,TT se comunica ya que no desea el dinero completo,2020-09-01 10:58:47,2020-09-01 11:55:08,78523,Phone,NaN,2020-09-09 18:26:05,Completed,Normal,Se contacta a TT ya que tenía dudas con el Abo...,Call,Outbound,c9251749-cbf2-4984-b83c-d517fc4c39cb,Call,20200910.0,120.0,2020-09-09 23:29:27,2020-09-09 23:27:27,83.0,2020-09-09 23:26:04,frida.delvalle,0.0,Customer Happiness,0.0,VOICE,c9251749-cbf2-4984-b83c-d517fc4c39cb,5.233350e+11,5.255416e+11,2020-09-09 23:27:28,2020-09-09 23:25:29,NaN,NaN,NaN,OUTBOUND,Customer Success,2020-09-10 03:00:42,"""connect-4ea90fb20d17/connect/connect-konfio/C...","[bien gracias, sabes qué, conseguir cosa yo só...",connect-4ea90fb20d17/connect/connect-konfio/Ca...,"[1, 1, 1, 1, 1, 1]",6,1.000000
2,1036900,CAYA7607122V4,1069028,1328804,43839,V,2020-07-06,Closed,Phone,TT se comunica ya que no desea el dinero completo,2020-09-01 10:58:47,2020-09-01 11:55:08,78523,Phone,NaN,2020-09-10 11:44:31,Completed,Normal,Se contacta a TT con éxito para explicar el Ab...,Call,Outbound,a4201c01-9bd0-49a8-9eff-48075d8de498,Call,20200911.0,48.0,2020-09-10 17:01:15,2020-09-10 17:00:27,957.0,2020-09-10 16:44:30,frida.delvalle,0.0,Customer Happiness,0.0,VOICE,a4201c01-9bd0-49a8-9eff-48075d8de498,5.233350e+11,5.255416e+11,2020-09-10 17:00:28,2020-09-10 16:43:59,NaN,NaN,NaN,OUTBOUND,Customer Success,2020-09-11 03:00:23,"""connect-4ea90fb20d17/connect/connect-konfio/C...","[None, None, None, None, Muy bien gracias Y tú...",connect-4ea90fb20d17/connect/connect-konfio/Ca...,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...",85,0.964706
3,273858,MAMC760324DN4,273540,1323378,43742,E,2020-06-25,Closed,Phone,Top Up,2020-07-13 11:41:45,2020-08-04 16:51:57,72050,Phone,NaN,2020-08-04 16:55:56,Completed,Normal,Se intenta contactar al TT sin éxito.,Call,Outbound,44450569-1a38-426e-a64a-2965178f4840,Call,20200825.0,3.0,2020-08-04 21:56:02,2020-08-04 21:55:59,3.0,2020-08-04 21:55:56,frida.delvalle,0.0,Customer Happiness,0.0,VOICE,44450569-1a38-426e-a64a-2965178f4840,5.255735e+11,5.255416e+11,2020-08-04 21:55:59,2020-08-04 21:55:22,NaN,NaN,NaN,OUTBOUND,Customer Success,2020-08-25 16:03:58,"""connect-4ea90fb20d17/connect/connect-konfio/C...",[grabe su mensaje despues del tono],connect-4ea90fb20d17/connect/connect-konfio/Ca..

**Interactions timeline**

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

In [7]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification',
                      model='joeddav/xlm-roberta-large-xnli')
LABELS='labels'
SCORES='scores'

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
labels=[ "positivo", "negativo", "neutro"]
res=classifier(['no estoy del lado de ningun bando'], labels,multi_class=False)
dict(zip(res[LABELS], res[SCORES]))

{'negativo': 0.005789698101580143,
 'neutro': 0.9753324389457703,
 'positivo': 0.0188779067248106}

In [9]:
for file in list(interactions_calls.keys()):
  interactions_calls[file].loc[:,'transcript']=interactions_calls[file].transcript.apply(lambda t: list(filter(None, t)) if t is not None else [])

In [10]:
def predict(row,labels=labels,classifier=classifier,multi_class=False):
    sentiments=[]
    if row[0]<1:
      #vad_n
      return None
    elif row[1]>0.6:
      #prob_speech
      for sentence in row[2]:
        #transcript
        res=classifier(sentence, labels,multi_class=multi_class)
        sentiments.append(dict(zip(res[LABELS], res[SCORES])))
    else:
      return None
    return {k: [dic[k] for dic in sentiments] for k in sentiments[0]}


In [11]:
from multiprocessing import Pool
from functools import partial 
predict_binary=partial(predict,labels=['positivo','negativo'],multi_class=True)
for file in interactions_calls.keys():
  transcripts=interactions_calls[file][['vad_n','prob_speech','transcript']].values
  with Pool(8) as p:
    interactions_calls[file]['sentiment']=[sentiments for sentiments in tqdm(p.imap(predict,transcripts),total=len(transcripts))]
  with Pool(8) as p:
    interactions_calls[file]['sentiment_binary']=[sentiments for sentiments in tqdm(p.imap(predict_binary,transcripts),total=len(transcripts))]

In [12]:
interactions_calls['opportunities_transcriptions_VM.pkl'][['transcript','sentiment','sentiment_binary']].head(3)

,transcript,sentiment,sentiment_binary
0,"[la gripa, Disculpa es que siempre andamos acá...","{'negativo': [0.5728643536567688, 0.1995569616...","{'negativo': [0.8840527534484863, 0.8491271734..."
1,"[la gripa, Disculpa es que siempre andamos acá...","{'negativo': [0.5728643536567688, 0.1995569616...","{'negativo': [0.8840527534484863, 0.8491271734..."
2,"[Christian Madrid, a ver si me explico has de ...","{'positivo': [0.42064476013183594, 0.544556736...","{'positivo': [0.886306881904602, 0.74713230133..."


In [13]:
samples=interactions_calls['cases_transcriptions_VM.pkl'][interactions_calls['cases_transcriptions_VM.pkl'].sentiment.notnull()][['transcript','sentiment','sentiment_binary']]

In [14]:
import plotly.graph_objects as go
def plot_lines(sample):
  fig = go.Figure()
  for key in sample.keys():
    fig.add_trace(go.Scatter(x=list(range(len(sample[key]))), y=sample[key], fill='tozeroy',name=key)) # fill down to xaxi
  fig.show()

In [15]:
for sample in samples.to_dict('records'):
  print(sample['transcript'])
  plot_lines(sample['sentiment'])
  plot_lines(sample['sentiment_binary'])

['la gripa', 'Disculpa es que siempre andamos acá', 'bajacalifornianos', 'átomos están hablando para querer', 'miren como rentas pues', 'según ellos para cuidarnos y no sé qué entonces', 'Aquí tengo aquí', 'a todos diciéndole si preguntan de dónde me hablas para que eres y todo pero bueno', 'dígame', 'Pues ojalá que se antes de lonche porque si no va a pasar lo mismo que el mes pasado Que se va a juntar el pago', 'muy amable Muchísimas gracias', 'buen día', 'igualmente Bye']


['bien gracias', 'sabes qué', 'conseguir cosa yo sólo así la sacamos', 'no puedo regresar la llamada como ahora no te puedo marcar para resolver la duda', 'después de las 9 de la mañana en el transcurso de la mañana Está bien', 'el estar triste en cualquier horario']


['Muy bien gracias Y tú', 'amino', 'que teníamos que el préstamo que nos hicieron perón 190', 'este Y regresamos $200000 que no vamos a utilizar', 'era regresar los 200 y aparte para la mensualidad', 'esas cositas nos tomará la mensualidad', 'es que según yo todavía no era hasta esperar La Primera mensualidad', 'según yo todavía no era fecha para usar la Primera mensualidad por eso solamente regresamos el día', 'ir a la fecha', 'Salazar apellido', 'a ver entonces tendría lazos 43-2000', 'depositarlos', 'mentalidad de pobre', 'ahorita con o', 'ocelote mexicano', 'la primer mensualidad', 'pero cuando yo le largues mentalidad me lo van a abonar esa mensualidad a capital', 'Pero no entiendo porque me lo quieren cobrar antes y la fecha', 'No pues eso nunca me dijeron cuando les dije que iba a regresar los $200000', 'Chicago estoy a llover a depositar hoy a decir ya que hacer', 'imagen oficial que teníamos que depositar', 'no se ha llegado la fecha', 'es hasta octubre', '943 $1000', 'antes d

['Real Madrid unas caras', 'de parte de quién', 'de qué producen el trabado', 'un poco trabajo', 'Pues bien gracias a Dios', 'yo nunca solicite', 'buen amor', 'Okay muy amable por su tiempo Muchas gracias']


['bueno', 'si soy yo', 'bien gracias', 'Es normal que solamente nuevo', 'no lo voy a usar todo', 'no molestar todo el dinero y solamente me quedo con la parte que te voy a necesitar', 'el día de hoy', 'tengo que volver a llamar después de hacer el depósito como hoy ya me espero', 'arroba konfio.mx', 'alirio', 'ya nomás Quedó en espera de la información', 'Gracias igualmente']


In [16]:
samples.loc[0,:]['transcript']

['la gripa',
 'Disculpa es que siempre andamos acá',
 'bajacalifornianos',
 'átomos están hablando para querer',
 'miren como rentas pues',
 'según ellos para cuidarnos y no sé qué entonces',
 'Aquí tengo aquí',
 'a todos diciéndole si preguntan de dónde me hablas para que eres y todo pero bueno',
 'dígame',
 'Pues ojalá que se antes de lonche porque si no va a pasar lo mismo que el mes pasado Que se va a juntar el pago',
 'muy amable Muchísimas gracias',
 'buen día',
 'igualmente Bye']

In [17]:
negativo=np.array(samples.loc[0,:]['sentiment_binary']['negativo'])
positivo=np.array(samples.loc[0,:]['sentiment_binary']['positivo'])

In [18]:
diff=positivo-negativo

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(diff))), y=diff,name='sentiment')) # fill down to xaxi
fig.show()

In [20]:
whatsapp=pd.read_csv(glob(f"{root_path}/*/*/*.csv")[0])

In [21]:
whatsapp=whatsapp[whatsapp.des_text.notna()]

In [22]:
wsample=whatsapp[(whatsapp.id==whatsapp.id.unique()[0])].sort_values(by=['date_created_at'])

In [23]:
text=wsample[['des_text','des_msg_for_out','id_message','date_created_at']]

In [24]:
client=text[(text.des_msg_for_out=='Cliente') & (text.des_text.str.split().str.len()>1)].copy(deep=True)

In [25]:
client.reset_index(inplace=True,drop=True)

In [26]:
client

,des_text,des_msg_for_out,id_message,date_created_at
0,No habra ningun apoyo?,Cliente,ABGHUhZlEQAgDwIKOgr5UYygfVLLSg,2020-04-07 15:48:58
1,Ok gracias Este mes no creo tener problema por...,Cliente,ABGHUhZlEQAgDwIKOtEzXUIGHqRZ_g,2020-04-07 22:23:48
2,Ok yo les aviso si necesitare dicha prorroga,Cliente,ABGHUhZlEQAgDwIKOlqcKhZqBJlM8A,2020-04-07 22:49:24
3,"Esos $3,370.38 como los pago?",Cliente,ABGHUhZlEQAgDwIKOjs2z7ddSxBxvA,2020-04-28 19:51:37
4,Ok a que banco es?,Cliente,ABGHUhZlEQAgDwIKOqBLDxUqdVNlWA,2020-04-28 19:52:54
5,Buenas tardes,Cliente,ABGHUhZlEQAgDwIKOsqR2Ek9m_U1mg,2020-04-28 22:56:37
6,Como quedaria mi pago con esta opcion,Cliente,ABGHUhZlEQAgDwIKOqMiJmZiQ_pzug,2020-04-28 22:56:49
7,Como quedaria mi opcion de pago anticipado ?,Cliente,ABGHUhZlEQAgDwIKOpIqEVjMkCpWqg,2020-04-29 22:56:46
8,Por favor,Cliente,ABGHUhZlEQAgDwIKOkB2e_hxJKGFmA,2020-04-29 22:56:49
9,Me eefiero al programa de pago anticipado,Cliente,ABGHUhZlEQAgDwIKOr5LBUASPwSUAA,2020-04-30 00:04:18


In [27]:
sentiments=classifier(client.des_text.values, labels,multi_class=True)

In [28]:
sentiments=[dict(zip(sample[LABELS],sample[SCORES])) for sample in sentiments]
sentiments={k: [dic[k] for dic in sentiments] for k in sentiments[0]}

In [29]:
sentiments_binary=classifier(client.des_text.values, labels[:-1],multi_class=False)
sentiments_binary=[dict(zip(sample[LABELS],sample[SCORES])) for sample in sentiments_binary]
sentiments_binary={k: [dic[k] for dic in sentiments_binary] for k in sentiments_binary[0]}

In [30]:
negativo=np.array(sentiments_binary['negativo'])
positivo=np.array(sentiments_binary['positivo'])
diff=positivo-negativo

In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(diff))), y=diff,name='sentiment')) # fill down to xaxi
fig.show()

In [32]:
plot_lines(sentiments)

In [33]:
classifier(' '.join(client.des_text.values), labels[:-1],multi_class=False)

{'labels': ['positivo', 'negativo'],
 'scores': [0.5416203141212463, 0.45837968587875366],
 'sequence': 'No habra ningun apoyo? Ok gracias Este mes no creo tener problema por que ya cuento con el dinero Pero me preocupa Mayo Ok yo les aviso si necesitare dicha prorroga Esos $3,370.38 como los pago? Ok a que banco es? Buenas tardes Como quedaria mi pago con esta opcion Como quedaria mi opcion de pago anticipado ? Por favor Me eefiero al programa de pago anticipado Quiero cotizar como me quedarian los pagos del programa de pago anticipado? Que buen servicio !!!! Para eso si bien oportunos y rapidos Pero para mi consulta no tuvieron ni un minuto'}

In [34]:
calls_history=interactions_calls['cases_transcriptions_VM.pkl'][(interactions_calls['cases_transcriptions_VM.pkl'].sentiment.notnull()) & (interactions_calls['cases_transcriptions_VM.pkl'].rfc=='CAYA7607122V4')]

In [35]:
hist={'positivo':[],'negativo':[],'neutro':[]}
for call in calls_history.loc[:,'sentiment'].values:
  hist['positivo'].append(np.array(call['positivo']).mean())
  hist['neutro'].append(np.array(call['neutro']).mean())
  hist['negativo'].append(np.array(call['negativo']).mean())

In [36]:
plot_lines(hist)

In [37]:
hist_b={'positivo':[],'negativo':[]}
for call in calls_history.loc[:,'sentiment_binary'].values:
  hist_b['positivo'].append(np.array(call['positivo']).mean())
  hist_b['negativo'].append(np.array(call['negativo']).mean())

In [38]:
plot_lines(hist_b)